In [1]:
""" !pip install torch>=1.6
!pip install torchvision
!pip install numpy
!pip install pandas
!pip install tqdm
!pip install tensorboardX>=1.14
!pip install scipy 
!pip install opencv-python
!pip install clean-fid """

' !pip install torch>=1.6\n!pip install torchvision\n!pip install numpy\n!pip install pandas\n!pip install tqdm\n!pip install tensorboardX>=1.14\n!pip install scipy \n!pip install opencv-python\n!pip install clean-fid '

In [2]:
model_name="sd-unsplash_5k_blur_61KS-model-control-lora"

In [3]:
import os
import shutil

def gather_images(model_name):
    # create folders to store the images, or clear them if they already exist
    for folder in ['original_images', 'generated_images']:
        if os.path.exists(f"{model_name}/{folder}"):
            # remove existing files in the folder
            for filename in os.listdir(f"{model_name}/{folder}"):
                file_path = os.path.join(f"{model_name}/{folder}", filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f'Failed to delete {file_path}. Reason: {e}')
        else:
            os.makedirs(f"{model_name}/{folder}")

    # walk through the folders and subfolders to access the images
    for dirpath, dirnames, filenames in os.walk(model_name):
        if 'saves' in dirpath:
            for filename in filenames:
                if 'original_image' in filename:
                    # get the datetime directory name
                    datetime_dir = os.path.basename(dirpath)
                    # copy original image to original_images folder and include datetime in name
                    shutil.copy(os.path.join(dirpath, filename), os.path.join(f"{model_name}/original_images", f"{datetime_dir}_{filename}"))
                elif 'generated_image' in filename:
                    # get the datetime directory name
                    datetime_dir = os.path.basename(dirpath)
                    # copy generated image to generated_images folder and include datetime in name
                    shutil.copy(os.path.join(dirpath, filename), os.path.join(f"{model_name}/generated_images", f"{datetime_dir}_{filename}"))

# usage
gather_images(model_name)

In [4]:
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import os
import numpy as np

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def make_dataset(dir):
    if os.path.isfile(dir):
        images = [i for i in np.genfromtxt(dir, dtype=np.str, encoding='utf-8')]
    else:
        images = []
        assert os.path.isdir(dir), '%s is not a valid directory' % dir
        for root, _, fnames in sorted(os.walk(dir)):
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(root, fname)
                    images.append(path)

    return images

def pil_loader(path):
    return Image.open(path).convert('RGB')

class BaseDataset(data.Dataset):
    def __init__(self, data_root, image_size=[256, 256], loader=pil_loader):
        self.imgs = make_dataset(data_root)
        self.tfs = transforms.Compose([
                transforms.Resize((image_size[0], image_size[1])),
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
        self.loader = loader

    def __getitem__(self, index):
        path = self.imgs[index]
        img = self.tfs(self.loader(path))
        return img

    def __len__(self):
        return len(self.imgs)

In [5]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def mae(input, target):
    with torch.no_grad():
        loss = nn.L1Loss()
        output = loss(input, target)
    return output


def inception_score(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    """Computes the inception score of the generated images imgs

    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)
    print(N)

    assert batch_size > 0
    assert N >= batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)

In [6]:
import argparse
from cleanfid import fid

src=f"{model_name}/original_images" #@param {type:"string"}
dst=f"{model_name}/generated_images" #@param {type:"string"}

fid_score=0
fid_score = fid.compute_fid(src, dst, num_workers=0)
is_mean, is_std = inception_score(BaseDataset(dst), cuda=True, batch_size=1, resize=True, splits=1)

print('FID: {}'.format(fid_score))
print('IS:{} {}'.format(is_mean, is_std))

with open(f"{model_name}/scores.txt", 'w') as f:
	# write the scores to the file
	f.write(f"FID score: {fid_score}\n")
	f.write(f"Inception score mean: {is_mean}\n")
	f.write(f"Inception score std: {is_std}\n")

compute FID between two folders
Found 14 images in the folder sd-unsplash_5k_blur_61KS-model-control-lora/original_images


FID original_images :   0%|          | 0/1 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [7]:
from PIL import Image
from torchvision.transforms import ToTensor

# Create an instance of the perceptual loss function
perceptual_loss = VGGPerceptualLoss(loss_func=EuclideanDistanceMean())


# Load your images
image1 = Image.open("/content/original_image.png")
image2 = Image.open("/content/generated_image_2.png")

# Transform your images to tensors
transform = ToTensor()
image1 = transform(image1).unsqueeze(0)  # Unsqueeze to add artificial first dimension
image2 = transform(image2).unsqueeze(0)  # Unsqueeze to add artificial first dimension

# Compute the perceptual loss
loss = perceptual_loss(image1, image2)

print("Perceptual loss:", loss.item())

NameError: name 'VGGPerceptualLoss' is not defined

In [9]:
import torch
import torchvision
import torch.nn as nn
class VGGPerceptualLoss(nn.Module):
    def __init__(self, resize=True,loss_func=nn.MSELoss()):
        super(VGGPerceptualLoss, self).__init__()
        self.loss_func = loss_func
        blocks = []
        blocks.append(torchvision.models.vgg16(pretrained=True).features[:4].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[4:9].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[9:16].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[16:23].eval())
        for bl in blocks:
            for p in bl:
                p.requires_grad = False
        self.blocks = torch.nn.ModuleList(blocks)
        self.transform = torch.nn.functional.interpolate
        self.mean = torch.nn.Parameter(torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1))
        self.std = torch.nn.Parameter(torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1))
        self.resize = resize

    def forward(self, input, target):
        if input.shape[1] != 3:
            input = input.repeat(1, 3, 1, 1)
            target = target.repeat(1, 3, 1, 1)
        input = (input-self.mean) / self.std
        target = (target-self.mean) / self.std
        if self.resize:
            input = self.transform(input, mode='bilinear', size=(224, 224), align_corners=False)
            target = self.transform(target, mode='bilinear', size=(224, 224), align_corners=False)
        loss = 0.0
        x = input
        y = target
        loss_function=self.loss_func
        for block in self.blocks:
            x = block(x)
            y = block(y)
            loss += loss_function(x, y)  #Can be any loss_function you choose
        return loss/len(self.blocks)

In [10]:
class EuclideanDistanceMean(nn.Module):
    def __init__(self):
        super(EuclideanDistanceMean, self).__init__()
        self.pairwise_distance = nn.PairwiseDistance()

    def forward(self, input, target):
        return self.pairwise_distance(input, target).mean()

In [8]:
!pip install sewar

In [11]:
from PIL import Image
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
import numpy as np
from torchvision.transforms import ToTensor

def calculate_score(org,blur):
	# Resize images to the same dimensions
	blur = blur.resize(org.size)

	# Convert the images to numpy arrays
	blur_arr = np.array(blur)
	org_arr = np.array(org)

	perceptual_loss = VGGPerceptualLoss(loss_func=EuclideanDistanceMean())

	transform = ToTensor()
	image1 = transform(org).unsqueeze(0)  # Unsqueeze to add artificial first dimension
	image2 = transform(blur).unsqueeze(0)  # Unsqueeze to add artificial first dimension

	quality_assessment = {}

	# Calculate the quality assessment metrics and store them in the dictionary
	quality_assessment["MSE"] = mse(blur_arr, org_arr)
	quality_assessment["RMSE"] = rmse(blur_arr, org_arr)
	quality_assessment["PSNR"] = psnr(blur_arr, org_arr)
	quality_assessment["SSIM"] = ssim(blur_arr, org_arr)
	quality_assessment["UQI"] = uqi(blur_arr, org_arr)
	quality_assessment["MSSSIM"] = msssim(blur_arr, org_arr)
	quality_assessment["ERGAS"] = ergas(blur_arr, org_arr)
	quality_assessment["SCC"] = scc(blur_arr, org_arr)
	quality_assessment["RASE"] = rase(blur_arr, org_arr)
	quality_assessment["SAM"] = sam(blur_arr, org_arr)
	quality_assessment["VIF"] = vifp(blur_arr, org_arr)
	quality_assessment["PD"] = perceptual_loss(image1, image2).item()

	return quality_assessment


In [12]:

# Path to the main folder containing subfolders
main_folder_path = "sd_controlnet_canny"

In [16]:
import os
from PIL import Image


# Initialize variables for average score calculation
total_quality_assessment = {}
total_num_subfolders = 0

# Iterate over the subfolders in the main folder
for subfolder_name in os.listdir(main_folder_path):
    subfolder_path = os.path.join(main_folder_path, subfolder_name)

    if os.path.isdir(subfolder_path):
        # Load the original image
        original_image_path = os.path.join(subfolder_path, "original_image.png")
        original_image = Image.open(original_image_path)
        original_image = original_image.convert('RGB')
        # Initialize variables for average score calculation per subfolder
        subfolder_total_score = {}
        subfolder_num_generated_images = {}

        # Iterate over the generated images in the current subfolder
        for file_name in os.listdir(subfolder_path):
            if file_name.startswith("generated_image_") and file_name.endswith(".png"):
                # Load the generated image
                generated_image_path = os.path.join(subfolder_path, file_name)
                generated_image = Image.open(generated_image_path)
                generated_image = generated_image.convert('RGB')
                # Calculate the quality assessment metrics for the current generated image
                image_quality_assessment = calculate_score(generated_image, original_image)

                # Update the total quality assessment scores for the subfolder
                for metric, value in image_quality_assessment.items():
                    if metric not in subfolder_total_score:
                        subfolder_total_score[metric] = value
                        subfolder_num_generated_images[metric]=0
                    else:
                        if isinstance(value, tuple):
                            subfolder_total_score[metric] = tuple(x + y for x, y in zip(subfolder_total_score[metric], value))
                        else:
                            subfolder_total_score[metric] += value

                    subfolder_num_generated_images[metric] += 1

        # Calculate the average scores for the current subfolder
        subfolder_average_score = {}
        for metric, value in subfolder_total_score.items():
            if metric not in subfolder_average_score:
                subfolder_average_score[metric]=value
            else:
                if isinstance(value, tuple):
                    subfolder_average_score[metric] = tuple(x / subfolder_num_generated_images[metric] for x in value)
                else:
                    subfolder_average_score[metric] = value / subfolder_num_generated_images[metric]

        # Update the total quality assessment scores across all subfolders
        for metric, value in subfolder_average_score.items():
            if metric not in total_quality_assessment:
                total_quality_assessment[metric]=value
            else:
                if isinstance(value, tuple):
                    total_quality_assessment[metric] = tuple(x + value[i] for i, x in enumerate(total_quality_assessment[metric]))
                else:
                    total_quality_assessment[metric] += value

        # Increment the count of subfolders
        total_num_subfolders += 1

        # Save the quality assessment dictionary in a file within the current subfolder
        quality_assessment_file_path = os.path.join(subfolder_path, "quality_assessment.txt")
        with open(quality_assessment_file_path, "w") as qa_file:
            for metric, value in subfolder_average_score.items():
                qa_file.write("{}: {}\n".format(metric, value))

# Calculate the average quality assessment scores across all subfolders
average_quality_assessment = {}
for metric, value in total_quality_assessment.items():
    if isinstance(value, tuple):
        average_quality_assessment[metric] = tuple(x / total_num_subfolders for x in value)
    else:
        average_quality_assessment[metric] = value / total_num_subfolders

# Save the average quality assessment dictionary in model_quality_assessment.txt in the main folder
model_quality_assessment_file_path = os.path.join(main_folder_path, "model_quality_assessment.txt")
with open(model_quality_assessment_file_path, "w") as model_qa_file:
    for metric, value in average_quality_assessment.items():
        model_qa_file.write("{}: {}\n".format(metric, value))